This notebook documents how the `wind_kelmarsh_2019.csv` file was calculated.
The source file `Turbine_Data_Kelmarsh_2_2019-01-01_-_2020-01-01_229.csv` is not part of draf since it has 150 MB.

In [1]:
files = {
    # 1: "Turbine_Data_Kelmarsh_1_2019-01-01_-_2020-01-01_228.csv",
    2: "Turbine_Data_Kelmarsh_2_2019-01-01_-_2020-01-01_229.csv",
    # 3: "Turbine_Data_Kelmarsh_3_2019-01-01_-_2020-01-01_230.csv",
    # 4: "Turbine_Data_Kelmarsh_4_2019-01-01_-_2020-01-01_231.csv",
    # 5: "Turbine_Data_Kelmarsh_5_2019-01-01_-_2020-01-01_232.csv",
    # 6: "Turbine_Data_Kelmarsh_6_2019-01-01_-_2020-01-01_233.csv",
}

import pandas as pd
from pathlib import Path
import numpy as np
from draf.helper import write

turbine_capacity = 2050  # kW see also https://www.wind-turbine-models.com/turbines/889-senvion-mm92

# downloaded Kelmarsh_SCADA_2019_3085.zip from https://doi.org/10.5281/zenodo.5841834
# print(Path(files[1]).read_text()[:1500])

In [2]:
def get_profile(turbine):
    df_orig = pd.read_csv(Path(files[turbine]), skiprows=9, index_col=0, parse_dates=True)
    return df_orig.loc["2019", "Energy Export (kWh)"].resample("H").sum().reset_index(drop=True) / turbine_capacity

In [3]:
ser = get_profile(turbine=2)

In [4]:
print(f"The capacity factor is {ser.mean():.2f}")

The capacity factor is 0.33


In [5]:
# convert from UTC to German winter time
ser = pd.Series(data=np.roll(ser, 1), index=ser.index, name="Power")

In [6]:
ser.mean()

0.3288368972045885

In [7]:
assert len(ser) == 8760

In [8]:
ser.describe()

count    8760.000000
mean        0.328837
std         0.297384
min         0.000000
25%         0.081951
50%         0.235610
75%         0.510732
max         1.001951
Name: Power, dtype: float64

In [9]:
write(ser, "2019_wind_kelmarsh2.csv")